In [2]:
import numpy as np

In [3]:
snippets = np.load("snippets.npy")
snippets = snippets[..., :-1]
snippets.shape

(10595, 10, 362)

In [4]:
targets = np.load("targets.npy")
targets.shape

(10595,)

In [6]:
X = snippets.reshape(-1, 10*362)
X.shape

(10595, 3620)

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
X_scaled = StandardScaler().fit_transform(X)
X_scaled.shape

(10595, 3620)

In [9]:
from sklearn.decomposition import PCA

In [10]:
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)
X_pca.shape

(10595, 10)

In [11]:
np.cumsum(pca.explained_variance_ratio_)

array([0.23726579, 0.39457701, 0.46557105, 0.52071828, 0.55959835,
       0.58831376, 0.61347098, 0.637752  , 0.65879173, 0.67489281])

In [70]:
# small dataset
np.random.seed(97)
small_idx = np.random.choice(len(X_pca), size=1500)

X_pca_small = X_pca[small_idx]
target_small = targets[small_idx]
X_pca_small.shape, target_small.shape
assert 1. in target_small

# Fit GPs

In [71]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [72]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
gp = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=10)

In [73]:
gp.fit(X_pca_small, target_small)

GaussianProcessClassifier(copy_X_train=True, kernel=1**2 * RBF(length_scale=1),
                          max_iter_predict=100, multi_class='one_vs_rest',
                          n_jobs=None, n_restarts_optimizer=10,
                          optimizer='fmin_l_bfgs_b', random_state=None,
                          warm_start=False)

In [74]:
gp.log_marginal_likelihood()

-26.092547575316093

In [75]:
gp.score(X_pca, targets)

0.9991505427088249

In [76]:
p = gp.predict(X_pca)

In [77]:
np.unique(p)

array([0])